In [ ]:
#!pip3 install tensorflow-addons
#!pip3 install json
#!pip3 install keras
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/ProjectDL
%cd training_set_task3
path_train = '/content/drive/MyDrive/ProjectDL/training_set_task3/'
path_test = '/content/drive/MyDrive/ProjectDL/test_set_task3/'

In [ ]:
import json
import cv2
import string
import numpy as np
import itertools
from keras.models import Model
from keras.layers import *
from keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, f1_score
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# Load training data from txt file
with open('/content/drive/MyDrive/ProjectDL/training_set_task3/training_set_task3.txt') as f:
    train_data = json.load(f)

# Initialize empty lists to store image, text, and label data
images,texts,labels = [],[],[]

# Iterate through training data and process image, text, and label data
for data in train_data:
    img = cv2.imread(path_train + data['image'])
    img = cv2.resize(img, (224, 224))
    images.append(img)
    text = data['text'].replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join(text.split()).lower()
    texts.append(text)
    labels.append(data['labels'])


In [ ]:
def one_hot_encode_labels(labels):
    all_labels = list(set(list(itertools.chain(*labels)))) + ['']
    one_hot_labels = np.zeros((len(labels), len(all_labels)))
    for i, label_list in enumerate(labels):
        if label_list:
            for label in label_list:
                one_hot_labels[i, all_labels.index(label)] = 1
        else:
            one_hot_labels[i, -1] = 1
    return one_hot_labels

In [ ]:
all_labels = list(set(list(itertools.chain(*labels)))) + ['']
one_hot_labels = one_hot_encode_labels(labels)
one_hot_labels = np.array(one_hot_labels)
print(one_hot_labels.shape)
print(one_hot_labels)

In [ ]:

# Save data to json file
t_data = {
    'images': images.tolist(),
    'texts': texts,
    'labels_onehot': one_hot_labels.tolist(),
    'labels_string': labels,
    'all_labels': all_labels
}

with open('/content/drive/MyDrive/ProjectDL/training_set_task3/training_task3.json', 'w') as f:
    json.dump(t_data, f)

# Load data from json file
with open('/content/drive/MyDrive/ProjectDL/training_set_task3/training_task3.json', 'r') as f:
    t_data = json.load(f)


train_images = np.array(t_data['images'])
train_texts = t_data['texts']
train_one_hot_labels = np.array(t_data['labels_onehot'])
train_labels = t_data['labels_string'] 
train_all_labels = t_data['all_labels'] 


print(train_images.shape)
print(train_images[1], train_labels[1])

In [ ]:
# Load testing data from txt file
with open('/content/drive/MyDrive/ProjectDL/test_set_task3/test_set_task3.txt') as f:
    test_data = json.load(f)

# Initialize empty lists to store image, text, and label data
test_images,test_texts,test_labels = [],[],[]

# Iterate through testing data and process image, text, and label data
for tdata in test_data:

    img = cv2.imread(path_test + tdata['image'])
    img = cv2.resize(img, (224, 224))
    test_images.append(img)
    text = tdata['text'].replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = ' '.join(text.split()).lower()
    test_texts.append(text)
    test_labels.append(tdata['labels'])

#print(test_labels)


In [ ]:
# Convert image data to numpy array
test_images = np.array(test_images)

# Create one-hot encoded labels
test_all_labels = list(set(list(itertools.chain(*test_labels))))
test_all_labels.append('')
test_one_hot_labels = []
test_one_hot_labels = one_hot_encode_labels(test_labels)
test_one_hot_labels = np.array(test_one_hot_labels)
print(test_one_hot_labels.shape)
print(test_one_hot_labels)

# Save data to json file
test_data = {
    'images': test_images.tolist(),
    'texts': test_texts,
    'labels_onehot': test_one_hot_labels.tolist(),
    'labels_string': test_labels,
    'all_labels': test_all_labels
}



In [ ]:



with open('/content/drive/MyDrive/ProjectDL/training_task3.json', 'w') as f:
    json.dump(test_data, f)

# Load data from json file
with open('/content/drive/MyDrive/ProjectDL/training_task3.json', 'r') as f:
    test_data = json.load(f)



In [ ]:
test_images = np.array(test_data['images'])
test_texts = test_data['texts']
test_one_hot_labels = np.array(test_data['labels_onehot'])


In [ ]:
test_labels = test_data['labels_string']
test_all_labels = test_data['all_labels']

num_classes = len(all_labels)
input_shape = images.shape[1:]

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
image_size = 100
patch_size = 5
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 2
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  
transformer_layers = 5
mlp_head_units = [2048, 1024] 
x_train=images
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)

data_augmentation.layers[0].adapt(x_train)
count=0

# code taken from keras documentation for vision transformers
def mlp(x, hidden_units, dropout_rate):
    global count
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu, name='feats'+str(count))(x)
        x = layers.Dropout(dropout_rate)(x)
        count+=1
    return x

class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


In [ ]:
image = x_train[np.random.choice(range(x_train.shape[0]))]
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    augmented = data_augmentation(inputs) # data augmentation wasn't used
    patches = Patches(patch_size)(augmented)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)
    j=0
    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.2
        )(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.2)
        encoded_patches = layers.Add()([x3, x2])
        j+=1

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.2)(representation)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.4)
    logits = layers.Dense(num_classes,activation='sigmoid')(features)
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


x_train=images
y_train=one_hot_labels
x_test=test_images
y_test=test_one_hot_labels
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)
    model.summary()
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=["categorical_accuracy"])
    history = model.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=5,validation_data=[x_test,y_test])
    return history
vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)



In [ ]:
plt.figure(figsize=(12,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.figure(figsize=(12,6))

plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])

plt.title('model acc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:

from keras.models import Model
# take features from the model
feature_network = Model(vit_classifier.input, vit_classifier.get_layer('feats13').output)
X_train = feature_network.predict(images)
X_valid = feature_network.predict(test_images)
y_train=one_hot_labels
y_valid=test_one_hot_labels
#print(X_valid.shape,y_valid.shape)

In [ ]:

# create an instance of RandomForestClassifier with 5 trees
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

# make predictions on the train and validation sets
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)

train_acc = accuracy_score(y_train, y_train_pred)
valid_acc = accuracy_score(y_valid, y_valid_pred)

print("Training accuracy: ", train_acc)
print("Validation accuracy: ", valid_acc)
print("F1 score: ", f1_score(y_valid, y_valid_pred, average='weighted'))

In [ ]:
clf = LogisticRegression(C = 4, penalty='l2', solver = 'liblinear', random_state=40)

model = OneVsRestClassifier(clf)
model.fit(X_train, y_train)

y_train_pred_proba = model.predict_proba(X_train)
y_valid_pred_proba = model.predict_proba(X_valid)

y_pred = (y_train_pred_proba > 0.3) 
y_train_new=(y_train > 0.3) 


y_pred1 = (y_valid_pred_proba > 0.3) 
y_valid_new=(y_valid > 0.3) 

print("Training accuracy: ", accuracy_score(y_train_new, y_pred))
print("Validation Accuracy: ", accuracy_score(y_valid_new, y_pred1))
print("F1 score: ", f1_score(y_valid_new,y_pred1,average='weighted'))